> **Rappel** : clique sur une cellule grise, puis **Shift + Entree** pour l'executer.
> Execute les cellules **dans l'ordre** de haut en bas.

---

# Leçon 2 : Apprendre de ses erreurs

## Le secret de l'IA : se tromper, corriger, recommencer

Imagine que tu apprends à lancer une balle dans un panier :
1. Tu lances -> tu rates à droite
2. Tu corriges un peu à gauche
3. Tu relances -> plus près !
4. Tu continues jusqu'à marquer

L'IA fait **exactement** pareil. Elle fait une prédiction, regarde si c'est
bon, et ajuste. Ça s'appelle **l'entraînement**.

In [ ]:
from IPython.display import HTML, display

_exercices_faits = set()
_NB_TOTAL = 3


def verifier(num_exercice, condition, message_ok, message_aide=""):
    """Valide un exercice avec feedback HTML vert/rouge + compteur."""
    try:
        _result = bool(condition)
    except Exception:
        _result = False
    if _result:
        _exercices_faits.add(num_exercice)
        n = len(_exercices_faits)
        barre = "\U0001f7e9" * n + "\u2b1c" * (_NB_TOTAL - n)
        display(
            HTML(
                f'<div style="padding:10px;background:#d4edda;border-left:5px solid #28a745;'
                f'margin:8px 0;border-radius:4px;font-family:system-ui,-apple-system,sans-serif">'
                f"\u2705 <b>{message_ok}</b><br>"
                f'<span style="color:#555">Progression : {barre} {n}/{_NB_TOTAL}</span></div>'
            )
        )
        if n == _NB_TOTAL:
            display(
                HTML(
                    '<div style="padding:12px;background:linear-gradient(135deg,#3949ab,#6a1b9a);'
                    "color:white;border-radius:8px;text-align:center;font-family:system-ui,-apple-system,sans-serif;"
                    'font-size:1.2em;margin:8px 0">\U0001f3c6 <b>Bravo ! Toutes les activites de cette lecon sont terminees !</b></div>'
                )
            )
    else:
        display(
            HTML(
                f'<div style="padding:10px;background:#fff3cd;border-left:5px solid #ffc107;'
                f'margin:8px 0;border-radius:4px;font-family:system-ui,-apple-system,sans-serif">'
                f"\U0001f4a1 <b>{message_aide}</b></div>"
            )
        )


def exercice(numero, titre, consigne, observation=""):
    """Affiche la banniere d'exercice."""

    def _style_code(text):
        return text.replace(
            "<code>",
            '<code style="font-size:0.95em;background:#bbdefb;'
            'padding:1px 5px;border-radius:3px;font-family:monospace;">',
        )

    obs = ""
    if observation:
        obs = (
            f'<div style="margin-top:6px;color:#555;font-size:0.92em;">'
            f"<b>Ce que tu vas voir\u00a0:</b> {_style_code(observation)}</div>"
        )
    display(
        HTML(
            f'<div style="border-left:5px solid #1565c0;background:#e8f0fe;'
            f"padding:12px 16px; margin:4px 0 10px 0; border-radius:0 8px 8px 0;"
            f'font-family:system-ui,-apple-system,sans-serif; font-size:0.95em;">'
            f'<b style="color:#0d47a1;">Exercice\u00a0{numero} \u2014 {titre}</b><br>'
            f"{_style_code(consigne)}{obs}</div>"
        )
    )


def afficher_evolution_loss(pertes, titre="Courbe de loss"):
    """Affiche la loss sous forme de barres verticales HTML avec degrade."""
    if not pertes:
        return
    max_loss = max(pertes)
    min_loss = min(pertes)
    bars = ""
    n = len(pertes)
    bar_w = max(4, min(20, 600 // n))
    for i, loss in enumerate(pertes):
        h = int((loss / max_loss) * 120) if max_loss > 0 else 0
        ratio = (loss - min_loss) / (max_loss - min_loss) if max_loss > min_loss else 0
        r = int(220 * ratio + 30)
        g = int(180 * (1 - ratio) + 50)
        bars += (
            f'<div style="display:inline-block;width:{bar_w}px;vertical-align:bottom;'
            f'margin:0 1px;height:{h}px;background:rgb({r},{g},60);border-radius:2px 2px 0 0" '
            f'title="Epoch {i}: {loss:.3f}"></div>'
        )
    display(
        HTML(
            f'<!-- tuto-viz --><div style="margin:8px 0"><b>{titre}</b>'
            f'<div style="display:flex;align-items:flex-end;height:140px;padding:8px;'
            f'background:#f8f9fa;border-radius:4px;margin-top:4px">{bars}</div>'
            f'<div style="display:flex;justify-content:space-between;color:#555;font-size:0.8em;margin-top:2px">'
            f"<span>Epoch 0 (loss={pertes[0]:.2f})</span><span>Epoch {len(pertes) - 1} (loss={pertes[-1]:.2f})</span></div></div>"
        )
    )


print("Outils de visualisation charges !")

---
## Étape 1 : Mesurer l'erreur

D'abord, il faut un moyen de dire **à quel point** le modèle s'est trompé.
On appelle ça la **loss** (perte en anglais).

- Loss haute = le modèle se trompe beaucoup
- Loss basse = le modèle devine bien

In [ ]:
import math

# Imaginons que le modèle prédit les probabilités suivantes
# pour la lettre qui suit 'p' dans 'pikachu' :

prediction = {
    "i": 0.6,  # 60% -> bonne réponse !
    "a": 0.2,  # 20%
    "e": 0.15,  # 15%
    "o": 0.05,  # 5%
}

# La bonne réponse est 'i'
bonne_reponse = "i"

# La loss = à quel point on est surpris par la bonne réponse
# Si on avait dit 100% pour 'i', la surprise serait de 0 (parfait !)
# Si on avait dit 1% pour 'i', la surprise serait énorme

loss = -math.log(prediction[bonne_reponse])
print(
    f"Le modèle donnait {prediction[bonne_reponse]:.0%} de chance à '{bonne_reponse}'"
)
print(f"Loss = {loss:.2f}")
print()

# Comparons avec une mauvaise prédiction
mauvaise_prediction = {"i": 0.05, "a": 0.7, "e": 0.2, "o": 0.05}
loss_mauvaise = -math.log(mauvaise_prediction[bonne_reponse])
print(
    f"Si le modèle n'avait donné que {mauvaise_prediction[bonne_reponse]:.0%} à '{bonne_reponse}'..."
)
print(f"Loss = {loss_mauvaise:.2f}  (beaucoup plus haut = beaucoup plus faux)")

In [ ]:
exercice(
    1,
    "Joue avec la perte",
    "Change <code>ma_prediction</code> ci-dessous (essaie 0.9 ou 0.01), puis <b>Shift + Entree</b>.",
    "Plus ta prediction est loin de la bonne reponse, plus la loss monte.",
)

# ==== MODIFIE ICI ====
ma_prediction = 0.6  # <-- Change cette valeur ! Essaie 0.9 ou 0.01
# ======================

ma_loss = -math.log(ma_prediction)
print(f"Si le modele donne {ma_prediction:.0%} de chance a la bonne reponse :")
print(f"  Loss = {ma_loss:.2f}")
if ma_prediction > 0.8:
    print("  -> Tres bien ! Le modele est confiant et a raison.")
elif ma_prediction < 0.1:
    print("  -> Enorme loss ! Le modele s'est beaucoup trompe.")
else:
    print("  -> Moyen. Le modele peut encore s'ameliorer.")

# Validation exercice 1
verifier(
    1,
    ma_prediction != 0.6,
    f"Bien joue ! Avec {ma_prediction:.0%}, la loss est {ma_loss:.2f}.",
    "Change ma_prediction pour une autre valeur, par exemple 0.9 ou 0.01.",
)

In [ ]:
# --- Étape 2 : Les poids du modèle ---
# Un modèle = une collection de nombres (les "poids").
# Ces nombres déterminent les prédictions. Entraîner = trouver les bons nombres.

import math
import random

# On crée un mini-modèle : juste des scores pour chaque paire de lettres
# Au début, les scores sont aléatoires -> le modèle ne sait rien

alphabet = list("abcdefghijklmnopqrstuvwxyz.")

# Scores aléatoires (les "poids" du modèle)
random.seed(42)
poids = {}
for a in alphabet:
    poids[a] = {}
    for b in alphabet:
        poids[a][b] = random.uniform(-1, 1)


def calculer_probas(poids, lettre):
    """Transforme les scores en probabilités (softmax)."""
    scores = poids[lettre]
    # L'exponentielle rend tous les scores positifs
    exps = {b: math.exp(scores[b]) for b in scores}
    total = sum(exps.values())
    return {b: exps[b] / total for b in scores}


# Au début, les probas sont quasi uniformes (le modèle devine au hasard)
p = calculer_probas(poids, ".")
lettres_debut = sorted(p.items(), key=lambda x: -x[1])[:5]
print("Au début, le modèle pense que les Pokémon commencent par :")
for lettre, prob in lettres_debut:
    print(f"  '{lettre}' : {prob:.1%}")
print("\n  -> C'est n'importe quoi ! Il faut l'entraîner.")

---
## Étape 3 : Entraînement

L'algorithme est simple :
1. Prendre un nom de Pokémon d'entraînement
2. Le modèle fait sa prédiction
3. On calcule la loss (l'erreur)
4. On **ajuste les poids** pour réduire la loss
5. Recommencer

L'étape 4 s'appelle la **descente de gradient**. C'est comme ajuster ton
tir au panier un petit peu à chaque essai.

In [ ]:
pokemons = [
    "arcanin",
    "bulbizarre",
    "carapuce",
    "dracaufeu",
    "ectoplasma",
    "evoli",
    "felinferno",
    "gardevoir",
    "goupix",
    "lokhlass",
    "lucario",
    "metamorph",
    "mewtwo",
    "noctali",
    "pikachu",
    "rondoudou",
    "ronflex",
    "salameche",
    "togepi",
    "voltali",
]

# Vitesse d'apprentissage : de combien on ajuste à chaque fois
# Trop grand = on dépasse, trop petit = on apprend trop lentement
vitesse = 0.1  # <-- Change cette valeur ! Essaie 0.01 ou 0.5
nb_epochs = 50  # <-- Change cette valeur ! Essaie 10 ou 200

print("Entraînement...")
print()

_historique_loss = []

for epoch in range(nb_epochs):
    loss_totale = 0
    nb = 0

    for pokemon in pokemons:
        mot = "." + pokemon + "."
        for i in range(len(mot) - 1):
            lettre = mot[i]
            cible = mot[i + 1]

            # 1. Prédiction
            probas = calculer_probas(poids, lettre)

            # 2. Loss
            loss_totale += -math.log(probas[cible] + 1e-10)
            nb += 1

            # 3. Ajuster les poids (gradient simplifié)
            for b in alphabet:
                if b == cible:
                    # La bonne réponse : augmenter son score
                    poids[lettre][b] += vitesse * (1 - probas[b])
                else:
                    # Les mauvaises réponses : baisser leur score
                    poids[lettre][b] -= vitesse * probas[b]

    _historique_loss.append(loss_totale / nb)

    if epoch % 10 == 0:
        print(f"  Epoch {epoch:2d} | Loss moyenne : {loss_totale / nb:.3f}")

print(f"  Epoch {epoch:2d} | Loss moyenne : {loss_totale / nb:.3f}")
print()
print("La loss baisse = le modèle s'améliore !")

# Visualisation de la courbe de loss
afficher_evolution_loss(
    _historique_loss, titre="Evolution de la loss pendant l'entrainement"
)

In [ ]:
exercice(
    2,
    "Compare les vitesses",
    "Change <code>vitesse_test</code> ci-dessous (essaie 0.01, 0.1, 0.5, 2.0).",
    "Une vitesse trop grande fait exploser la loss, trop petite la fait stagner.",
)

# ==== MODIFIE ICI ====
vitesse_test = 0.5  # <-- Change cette valeur ! Essaie 0.01, 0.1, 0.5, 2.0
# ======================

# On repart de zero avec des poids aleatoires
random.seed(123)
poids_test = {}
for a in alphabet:
    poids_test[a] = {}
    for b in alphabet:
        poids_test[a][b] = random.uniform(-1, 1)

# Mini-entrainement de 5 epochs avec cette vitesse
for epoch in range(5):
    loss_t = 0
    nb_t = 0
    for pokemon in pokemons:
        mot = "." + pokemon + "."
        for i in range(len(mot) - 1):
            probas_t = calculer_probas(poids_test, mot[i])
            loss_t += -math.log(probas_t[mot[i + 1]] + 1e-10)
            nb_t += 1
            for b in alphabet:
                if b == mot[i + 1]:
                    poids_test[mot[i]][b] += vitesse_test * (1 - probas_t[b])
                else:
                    poids_test[mot[i]][b] -= vitesse_test * probas_t[b]
    print(f"  Epoch {epoch} | vitesse={vitesse_test} | Loss : {loss_t / nb_t:.3f}")

# Validation exercice 2
verifier(
    2,
    vitesse_test != 0.5,
    f"Bien ! Tu as teste la vitesse {vitesse_test}.",
    "Change vitesse_test pour une autre valeur, par exemple 0.01 ou 2.0.",
)

In [ ]:
# Voyons maintenant ce que le modèle a appris :
p = calculer_probas(poids, ".")
lettres_debut = sorted(p.items(), key=lambda x: -x[1])[:5]
print("Après entraînement, les Pokémon commencent par :")
for lettre, prob in lettres_debut:
    print(f"  '{lettre}' : {prob:.1%}")
print()
print("C'est plus logique ! (c, m, g, e, r sont des débuts courants)")

In [ ]:
# Générons des noms de Pokémon avec le modèle entraîné
def generer(poids, n=10):
    resultats = []
    for _ in range(n):
        pokemon = ""
        lettre = "."
        for _ in range(20):  # max 20 lettres
            p = calculer_probas(poids, lettre)
            choix = list(p.keys())
            probs = list(p.values())
            lettre = random.choices(choix, weights=probs, k=1)[0]
            if lettre == ".":
                break
            pokemon += lettre
        if pokemon:
            resultats.append(pokemon.capitalize())
    return resultats


print("Noms de Pokémon inventés après entraînement :")
for p in generer(poids, 10):
    print(f"  {p}")

In [ ]:
exercice(
    3,
    "Genere des Pokemon",
    "Change <code>nombre</code> ci-dessous (essaie 50).",
    "Certains noms vont ressembler a de vrais Pokemon !",
)

# ==== MODIFIE ICI ====
nombre = 10  # <-- Mets 50 ici !
# ======================

print(f"Generation de {nombre} Pokemon :")
print()
for i, nom in enumerate(generer(poids, nombre)):
    print(f"  {i + 1}. {nom}")

# Validation exercice 3
verifier(
    3,
    nombre != 10,
    f"Genial ! Tu as genere {nombre} Pokemon.",
    "Change nombre pour une autre valeur, par exemple 50.",
)

---
## Ce qu'on a appris

- La **loss** mesure à quel point le modèle se trompe
- Les **poids** sont les nombres que le modèle ajuste pour apprendre
- L'**entraînement** = ajuster les poids pour réduire la loss, encore et encore
- Même un modèle simple s'améliore avec l'entraînement !

### Limite

Notre modèle ne regarde encore que **1 lettre en arrière**.
Dans la prochaine leçon, on va lui donner une **mémoire** pour qu'il
se souvienne de plusieurs lettres à la fois.

---
*Prochaine leçon : [03 - La mémoire du modèle](03_la_memoire_du_modele.ipynb)*

---

### Sources (ISO 42001)

- **Cross-entropy loss et descente de gradient** : [microgpt.py](https://gist.github.com/karpathy/8627fe009c40f57531cb18360106ce95) — Andrej Karpathy, lignes implémentant le backward pass
- **Analogie du gradient comme correction** : [Vidéo "Let's build GPT"](https://www.youtube.com/watch?v=kCc8FmEb1nY) — Andrej Karpathy (2023)
- **Visualisation de la descente de gradient** : [3Blue1Brown - Gradient descent](https://www.youtube.com/watch?v=IHZwWFHWa-w) — Grant Sanderson
- **Dataset Pokémon** : (c) Nintendo / Creatures Inc. / GAME FREAK inc., usage éducatif. Source : [PokéAPI](https://pokeapi.co/)